UBER almacena en un cluster todos los datos sobre el movimiento y
viajes de todos sus vehículos. Existe un proceso que nos devuelve un
RDD llamado trip_summary con los siguientes campos: (driver_id,
car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled),
Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para
Abril de 2016.  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqs4v2c4xx)

Vamos a generar una lista de tuplas con `faker` para las fechas

In [ ]:
from faker import Faker
from datetime import date as dt
from random import random, choice
fake = Faker()
csv = []

for _ in range(1000):
    fecha = fake.date_between_dates(date_start=dt(2016,1,1),date_end=dt(2016,12,31)).strftime("%Y%m%d")
    driver_id = fake.pyint() % 50
    car_id = fake.pyint() % 50
    trip_id = fake.pyint()
    customer_id = fake.pyint()
    distance_traveled = fake.pyfloat(positive=True) % 1000.0
    csv.append((driver_id, car_id, trip_id, customer_id, fecha, distance_traveled))

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

rdd = sc.parallelize(csv)

Primero filtramos los registros de Abril de 2016

In [ ]:
rdd = rdd.filter(lambda x: x[-2].startswith("201604") )

Como queremos el promedio, necesitamos saber cuantos viajes hizo y la suma por driver_id. Entonces tenemos que agrupar por driver_id, sumando las distancias recorridas y sumando 1 por cada viaje.  
Entonces primero con `map` preparamos una tupla con `(driver_id, distancia, 1)`. Luego agrupamos y sumamos con `reduceByKey`.

In [ ]:
rdd = rdd.map(lambda x: (x[0],(x[-1],1)))
rdd = rdd.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))

Para cada conductor calculamos el promedio haciendo `suma de distancias / cantidad de viajes`. 

In [ ]:
rdd = rdd.map(lambda x: (x[0],x[1][0]/x[1][1]))

Usamos `rdd.max` para obtener el registro con mayor promedio.

In [ ]:
conductor = rdd.max(key=lambda x:x[1])
"El conductor {} tuvo el mayor promedio: {}".format(conductor[0],conductor[1])